### HTML 데이터 파싱을 이용한 크롤링
네이버 연관 검색어
- 1. 웹 페이지 분석 : URL 찾기
- 2. 요청 -> 응답 : HTML(str)
- 3. HTML(str) -> BeautifulSoup obj (css selector) -> DF

In [21]:
import requests
from bs4 import BeautifulSoup

In [22]:
#1. 웹 페이지 분석 : URL
query = '삼성전자'
url = 'https://search.naver.com/search.naver?\
sm=top_hty&fbm=0&ie=utf8&query={}'.format(query)
print(url)

https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query=삼성전자


In [23]:
#2. request -> response : HTML(str)
response = requests.get(url)
response

<Response [200]>

In [24]:
#3. HTML(str) -> BeaBeautifulSoup Ibj (css selector) -> DF
dom = BeautifulSoup(response.content, 'html.parser')
type(dom)

bs4.BeautifulSoup

In [25]:
# dom
# select : 여려 개의 element 객체를 리스트로 가져옴
# select_one : 하나의 element 객체를 가져옴

In [26]:
elements = dom.select('._related_keyword_ul > li')
len(elements)

10

In [27]:
keywords = [element.text.strip() for element in elements]
print(keywords)

['삼성', '삼성물산', '삼성전자우', '삼성주식', '삼성생명', '카카오', '이건희', '빅히트', '삼성 주가', '현대자동차']


In [14]:
type(elements[0])

bs4.element.Tag

In [29]:
# 함수로 만들기
def naver_relational_keywords(query):
    url = 'https://search.naver.com/search.naver?\
sm=top_hty&fbm=0&ie=utf8&query={}'.format(query)
    response = requests.get(url)
    dom = BeautifulSoup(response.content, 'html.parser')
    elements = dom.select('._related_keyword_ul > li')
    return [element.text.strip() for element in elements]

In [30]:
query = '코로나'
result = naver_relational_keywords(query)
print(result)

['코로나 2.5단계 연장', '코로나 라이브', '코로나 백신', '코로나 확진자', '코로나 1단계', '코로나 2단계', '코로나 부산', '부산 코로나', '독감주사 사망', '코로나 증상']


#### 다음 뉴스 데이터
- 다음 뉴스의 메인 페이지 15개 기사의 제목, 링크, 내용 수집

In [80]:
#1. URL
url = 'https://news.daum.net/'

In [81]:
#2. HTML(str)
response = requests.get(url)
response

<Response [200]>

In [82]:
#3. BeautiBeautifulSoup
dom = BeautifulSoup(response.content, 'html.parser')

In [83]:
elements = dom.select('.list_headline > li')
len(elements)

15

In [84]:
elements[0]

<li class="item_main">
<a class="link_cont" data-tiara-custom="contentUniqueKey=hamny-20201027133917659" data-tiara-id="20201027133917659" data-tiara-layer="photo" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://news.v.daum.net/v/20201027133917659">
<span class="wrap_thumb">
<img alt="금감원 고질병?..비위 직원 잇단 '솜방망이' 처벌 논란" class="thumb_g" src="https://img1.daumcdn.net/thumb/S189x104ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202010%2F27%2FNEWS1%2F20201027133923004zqmr.jpg&amp;scode=media"/>
</span>
<strong class="tit_g">금감원 고질병?..비위 직원 잇단 '솜방망이' 처벌 논란</strong>
</a>
</li>

In [85]:
datas = []

for element in elements:
    datas.append({
        'title' : element.select_one('.tit_g').text.strip().replace('\n', ''),
        'link' : element.select_one('a').get('href'),
    })
    
articles_df = pd.DataFrame(datas)
articles_df.tail()

,title,link
10,"백신 속 백색입자, 원성분 단백질 불과..""효과·안전성 문제없다""",https://news.v.daum.net/v/20201027133640580
11,LG-SK 배터리 영업비밀 소송 최종 결정 또 연기한겨레,https://news.v.daum.net/v/20201027131605875
12,"""이혼소송 중에도 폭력을 휘두르는 남편에게서 어떻게 벗어날까요?""YTN",https://news.v.daum.net/v/20201027131527864
13,"김근식 ""영락없는 코미디..무개념 김김 헛소리에 조국 잽싸게 화답""중앙일보",https://news.v.daum.net/v/20201027131011734
14,"존재감 키운 윤석열, '대망론'에 쏟아진 말말말시사저널",https://news.v.daum.net/v/20201027130203526


In [86]:
# contect data 추가

In [87]:
link = articles_df.loc[0, 'link']
print(link)

https://news.v.daum.net/v/20201027133917659


In [90]:
response = requests.get(link)
dom = BeautifulSoup(response.content, 'html.parser')

In [93]:
content = dom.select_one('#harmonyContainer').text.strip().replace('\n', '')
content [:100]

"(서울=뉴스1) 박응진 기자 = 금융감독원이 비위를 일으킨 내부 직원들을 잇달아 '솜방망이' 처벌하는 등 내부통제에 실패한 결과에 대해 스스로 너무 관대한 것 아니냐는 지적이 제기"

In [94]:
content = dom.select_one('.summary_view').text.strip().replace('\n', '')
content [:100]

'유흥업소서 라임정보 건넸는데 포상전력 있다고 감봉으로 낮춰윤석헌 "직접 연루 없다"..권은희 "금감원, 형사 책임 당사자"'

In [99]:
# 함수로 만들기
def get_content(link):
    response = requests.get(link)
    dom = BeautifulSoup(response.content, 'html.parser')
    return dom.select_one('#harmonyContainer').text.strip().replace('\n', '')

In [101]:
articles_df["contents"] = articles_df['link'].apply(get_content)
articles_df.tail()

,title,link,contents
10,"백신 속 백색입자, 원성분 단백질 불과..""효과·안전성 문제없다""",https://news.v.daum.net/v/20201027133640580,정부가 만 62~69세를 대상으로 인플루엔자(독감) 백신 무료접종을 시작한 26일 ...
11,LG-SK 배터리 영업비밀 소송 최종 결정 또 연기한겨레,https://news.v.daum.net/v/20201027131605875,엘지(LG)화학과 에스케이(SK)이노베이션 간 배터리 영업비밀 침해 소송의 최종결정...
12,"""이혼소송 중에도 폭력을 휘두르는 남편에게서 어떻게 벗어날까요?""YTN",https://news.v.daum.net/v/20201027131527864,YTN라디오(FM 94.5) [양소영 변호사의 상담소] □ 방송일시 : 2020년 ...
13,"김근식 ""영락없는 코미디..무개념 김김 헛소리에 조국 잽싸게 화답""중앙일보",https://news.v.daum.net/v/20201027131011734,국민의힘 서울 송파병 당협위원장인 김근식 경남대 교수가 27일 김용민 더불어민주당 ...
14,"존재감 키운 윤석열, '대망론'에 쏟아진 말말말시사저널",https://news.v.daum.net/v/20201027130203526,(시사저널=이혜영 기자) 윤석열 검찰총장이 22일 서울 여의도 ...
